<a href="https://colab.research.google.com/github/DevPatel124/CRUD-IN-JS/blob/main/QualificationAnswers2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import sqlite3

In [7]:
conn_countries = sqlite3.connect("countries1.sql")
conn_languages = sqlite3.connect("languages1.sql")
excel_file = 'QualificationAnswers.xlsx'

In [8]:
for filename in ["countries1.sql", "languages1.sql"]:
    with open(filename, "r") as f:
        sql_statements = f.read().split(";")

    conn = conn_countries if filename == "countries1.sql" and filename == "languages1.sql" else conn_languages

    for statement in sql_statements:
        statement = statement.strip()  # Remove whitespaces
        if statement and not statement.startswith("--"):  # Skip empty lines and comments
            if statement.lower().startswith("set"):  # Handle various SET statements
                if "time_zone" in statement:
                    conn.execute("PRAGMA timezone = '+00:00'")
                elif "names" in statement:
                    pass
                else:
                    print(f"Skipping unsupported SET statement: {statement}")  # Log other SET statements
            elif statement.lower().startswith("start transaction"):
                conn.execute("BEGIN")
            else:
                conn.execute(statement)

In [ ]:
language_codes = pd.read_sql("SELECT languageName, languageCode2 FROM languages1", conn_languages)
country_codes = pd.read_sql("SELECT countries_name, countries_iso_code_2 FROM countries1", conn_countries)

In [ ]:
writer = pd.ExcelWriter(excel_file, engine='openpyxl', mode='a')

In [ ]:
for sheet_name in pd.ExcelFile(excel_file).sheet_names:
    df = pd.read_excel(excel_file, sheet_name=sheet_name)

    language, country = sheet_name.split('-')
    language_code = language_codes.get(language.title())  # Get language code in uppercase
    country_code = country_codes.get(country.title())  # Get country code in uppercase

    if language_code and country_code:
        language_code = f"{language_code}-{country_code}"
        if 'Language code' not in df.columns:
            df['Language code'] = language_code
        else:
            print(f"Skipping sheet '{sheet_name}' as it already has 'Language code' column")

        # Write updated DataFrame back to Excel
        df.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
writer.save()